# Check Data Compliance 

In [1]:
import pandas as pd
import datetime as dt
from datetime import date

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")

week_ago = week_ago.strftime("%Y-%m-%d")

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"
filepath = datapath + f"export_{today}.csv"


In [2]:
df = pd.read_csv(filepath, encoding= "latin-1")

In [3]:
df.drop(["Quelle", "Created At"], axis=1, inplace=True)

In [4]:
df["Endkunde"] = df.Endkunde.str.split("@").str.get(0)
df["Endkunde"] = df["Endkunde"].str[:4]
df.rename(columns={"Endkunde":"customer","Tag":"day", "Typ":"type","Wert(Fließkommazahl)": "float", "Wert(Ganzzahl)": "int"}, inplace=True)

In [5]:
df["day"] = pd.to_datetime(df["day"], format="%d.%m.%Y")

In [6]:
df["int"].fillna(df["float"], inplace=True)

In [7]:
df_int= df[["customer", "day", "type", "int"]]

In [8]:
df_int = df_int.pivot(
    index=["customer", "day"],
    columns="type",
    values="int")

In [9]:
df_int.reset_index(level="day", inplace=True)

In [10]:
df_active = pd.read_csv("/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/active_participants.csv")

In [11]:
df_active.rename(columns = {"Pseudonym ": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

In [12]:
df_active["customer"] = df_active["customer"].str[:4]

In [13]:
df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]

In [14]:
df_active.groupby("status").count()

,customer,ema_id,end_ema,start_ema,study_version,for_id
status,,,,,,
Abgeschlossen,15,15,15,15,15,15
Dropout,4,4,4,4,4,4
Erhebung_1_aktiv,12,12,12,12,12,12
Post_Erhebung_1,38,38,38,38,38,38


In [15]:
filepath_connection = f"/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/export_connection_{today}.csv"
df_con = pd.read_csv(filepath_connection, encoding= "latin-1")
df_con["Customer"] = df_con.Customer.str.split("@").str.get(0)
df_con["Customer"] = df_con["Customer"].str[:4]
df_con.drop(["Data Source", "Timestamp"], axis=1, inplace=True)
df_con.rename(columns = {"Customer": "customer"}, inplace=True)

In [16]:
df_complete = pd.merge(df_int, df_active, on="customer")

In [17]:
df_complete = pd.merge(df_complete, df_con, on="customer")

In [18]:
grouped = df_complete.groupby(["customer"])["ema_id","for_id","start_ema","end_ema","study_version",
                                            "status","Is Connected","day"].max()

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [19]:
grouped_count = df_complete.groupby(["customer"])[["day"]].count()
grouped_count.rename(columns={"day": "nr_days"}, inplace= True)
grouped = pd.merge(grouped, grouped_count, how= "left", on="customer")
grouped

,ema_id,for_id,start_ema,end_ema,study_version,status,Is Connected,day,nr_days
customer,,,,,,,,,
0ePW,EMA4006,FOR14007,03.07.2023,17.07.2023,Lang,Post_Erhebung_1,ja,2023-09-21,80
0xWn,EMA2004,FOR12003,27.07.2023,10.08.2023,Lang,Post_Erhebung_1,ja,2023-09-26,61
1rfO,EMA3006,FOR13002,27.06.2023,11.07.2023,Lang (Wechsel),Post_Erhebung_1,ja,2023-07-05,8
3ZqH,EMA3002,FOR13901\n,20.06.2023,04.07.2023,Kurz,Abgeschlossen,ja,2023-07-09,19
3s90,EMA3004,FOR13001,27.06.2023,11.07.2023,Kurz,Abgeschlossen,nein,2023-06-28,2
...,...,...,...,...,...,...,...,...,...
uN1K,EMA4009,FOR14011,12.07.2023,26.07.2023,Kurz (Wechsel/Abbruch),Abgeschlossen,ja,2023-09-14,28
vPEr,EMA4010,FOR14013,10.08.2023,24.08.2023,Kurz,Abgeschlossen,ja,2023-09-18,39
wGQ8,EMA1005,FOR11012,21.06.2023,05.07.2023,Lang (Wechsel),Post_Erhebung_1,ja,2023-09-07,60


In [20]:
grouped.groupby("status")["ema_id"].count()

status
Abgeschlossen       15
Dropout              3
Erhebung_1_aktiv    11
Post_Erhebung_1     38
Name: ema_id, dtype: int64

In [21]:
grouped_active = grouped[~(grouped['status'] == "Abgeschlossen")]
grouped_active = grouped_active[~(grouped_active['status'] == "Dropout")]

## GPS Data

In [22]:
datapath1 = datapath + f"export_tiki_{today}/"
filepath_1 = datapath1 + "epoch_part0001.csv"
filepath_2 = datapath1 + "epoch_part0002.csv"
filepath_3 = datapath1 + "epoch_part0003.csv"

In [23]:
df_1 = pd.read_csv(filepath_1, encoding= "latin-1")
df_2 = pd.read_csv(filepath_2, encoding= "latin-1")
df_3 = pd.read_csv(filepath_3, encoding= "latin-1")


/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
df_location_1 = df_1[df_1.type.isin(["Latitude", "Longitude"])]
df_location_2 = df_2[df_2.type.isin(["Latitude", "Longitude"])]
df_location_3 = df_3[df_3.type.isin(["Latitude", "Longitude"])]

In [25]:
df_loc_complete = pd.concat([df_location_1, df_location_2, df_location_3])
df_loc_complete = df_loc_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           "timezoneOffset"]]
df_loc_complete["customer"] = df_loc_complete.customer.str.split("@").str.get(0)
df_loc_complete["customer"] = df_loc_complete.customer.str[:4]

In [26]:
df_merged = pd.merge(df_loc_complete, grouped_active, on="customer")
df_merged["startTimestamp"] = (pd.to_datetime(df_merged["startTimestamp"],unit='ms'))
#df_merged_active = df_merged[~(df_merged['status'].isin(["Abgeschlossen", "Dropout"]))]

In [27]:
df_merged_active= df_merged.groupby("customer")[["ema_id","for_id","day","study_version","status","startTimestamp"]].max()

In [28]:
df_merged_active

,ema_id,for_id,day,study_version,status,startTimestamp
customer,,,,,,
0ePW,EMA4006,FOR14007,2023-09-21,Lang,Post_Erhebung_1,2023-09-24 21:45:59
0xWn,EMA2004,FOR12003,2023-09-26,Lang,Post_Erhebung_1,2023-09-25 20:00:35
1rfO,EMA3006,FOR13002,2023-07-05,Lang (Wechsel),Post_Erhebung_1,2023-09-25 21:48:49
4HX0,EMA1015,FOR11024,2023-09-25,Lang,Post_Erhebung_1,2023-09-18 14:21:34
5MyR,EMA4026,FOR14022,2023-09-26,Lang,Erhebung_1_aktiv,2023-09-25 21:18:58
7uRT,EMA4030,FOR14024,2023-09-25,Lang,Erhebung_1_aktiv,2023-09-22 20:58:55
8ear,EMA1004,FOR11005,2023-09-22,Lang,Post_Erhebung_1,2023-09-21 08:44:57
94FE,EMA2002,FOR12901,2023-09-26,Lang,Post_Erhebung_1,2023-09-25 16:23:27
AG7S,EMA1014,FOR11022,2023-09-26,Lang,Post_Erhebung_1,2023-08-28 08:09:35


In [29]:
df_merged_active.loc[(df_merged_active.startTimestamp < week_ago) & (df_merged_active.day < week_ago)]

,ema_id,for_id,day,study_version,status,startTimestamp
customer,,,,,,
TuLo,EMA3005,FOR13009,2023-09-12,Lang,Post_Erhebung_1,2023-08-24 12:13:09
wGQ8,EMA1005,FOR11012,2023-09-07,Lang (Wechsel),Post_Erhebung_1,2023-09-04 16:14:46


In [30]:
# no gpspd
df_merged_active.loc[(df_merged_active.startTimestamp < week_ago) & (df_merged_active.day < week_ago)]["for_id"].tolist()

['FOR13009', 'FOR11012']

In [31]:
# no gps
df_merged_active.loc[(df_merged_active.startTimestamp < week_ago) & (df_merged_active.day > week_ago)]

,ema_id,for_id,day,study_version,status,startTimestamp
customer,,,,,,
4HX0,EMA1015,FOR11024,2023-09-25,Lang,Post_Erhebung_1,2023-09-18 14:21:34
AG7S,EMA1014,FOR11022,2023-09-26,Lang,Post_Erhebung_1,2023-08-28 08:09:35
dT0F,EMA3003,FOR13008,2023-09-26,Lang,Post_Erhebung_1,2023-09-16 23:40:17
fjGD,EMA3007,FOR13011,2023-09-26,Lang,Post_Erhebung_1,2023-09-02 15:54:05
uGoA,EMA3012,FOR13005,2023-09-26,Lang,Post_Erhebung_1,2023-08-09 08:55:01


In [32]:
# no gps
df_merged_active.loc[(df_merged_active.startTimestamp < week_ago) & (df_merged_active.day > week_ago)]["for_id"].tolist()

['FOR11024 ', 'FOR11022', 'FOR13008', 'FOR13011', 'FOR13005']

In [33]:
# no gps
df_merged_active.loc[(df_merged_active.startTimestamp < week_ago) & (df_merged_active.day > week_ago)]["ema_id"].tolist()

['EMA1015', 'EMA1014', 'EMA3003', 'EMA3007', 'EMA3012']

In [34]:
# no pd
df_merged_active.loc[(df_merged_active.startTimestamp > week_ago) & (df_merged_active.day < week_ago)]

,ema_id,for_id,day,study_version,status,startTimestamp
customer,,,,,,
1rfO,EMA3006,FOR13002,2023-07-05,Lang (Wechsel),Post_Erhebung_1,2023-09-25 21:48:49
gfo4,EMA4003,FOR14002,2023-09-14,Lang (Wechsel),Post_Erhebung_1,2023-09-25 10:47:29
kH1x,EMA4002,FOR14003,2023-09-13,Lang,Post_Erhebung_1,2023-09-25 15:24:16


In [35]:
# no pd
df_merged_active.loc[(df_merged_active.startTimestamp > week_ago) & (df_merged_active.day < week_ago)]["for_id"].tolist()

['FOR13002', 'FOR14002', 'FOR14003']

## Missing data

In [36]:
list_1 = df_active.customer.tolist()
list_2 = df_complete.customer.unique().tolist()
set(list_1) - set(list_2)

{'Nuz5', 'x3TJ'}